In [1]:
import requests
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
classes = pd.read_csv("all_classes.csv")
classes[['Prefix', 'Course Number']] = classes['Course ID'].str.extract(r'([A-Za-z]+)(\d+)')



In [3]:
classes

,Unnamed: 0,Course ID,Campus,Department Name,College,Course Number,Course Name,Credits,Areas of Knowledge,Quarters Offered,Offered with,Prerequisites,Co-Requisites,Description,Prefix
0,3165,SWA101,Seattle,SWA,College of Arts and Sciences,101,Basic Swahili,5,NaN,NaN,NaN,NaN,NaN,Introduces the Swahili language and the divers...,SWA
1,3166,SWA102,Seattle,SWA,College of Arts and Sciences,102,Basic Swahili,5,NaN,NaN,NaN,SWA101,NaN,Introduces the Swahili language and the divers...,SWA
2,3167,SWA103,Seattle,SWA,College of Arts and Sciences,103,Basic Swahili,5,NaN,NaN,NaN,SWA102,NaN,Introduces the Swahili language and the divers...,SWA
3,3168,SWA134,Seattle,SWA,College of Arts and Sciences,134,Intensive Basic Swahili,15,NaN,S,NaN,NaN,NaN,Introduces the Swahili language and the divers...,SWA
4,3169,SWA201,Seattle,SWA,College of Arts and Sciences,201,Intermediate Swahili,5,NaN,NaN,NaN,"SWA103,SWA134",NaN,Builds proficiency in the language by speaking...,SWA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15677,18842,SOCW597,Seattle,SOCW,Social Work,597,Graduate Study Abroad in Social Work,"1-15, max. 15",NaN,NaN,NaN,NaN,NaN,For graduate students in a UW study abroad pro...,SOCW
15678,18843,SOCW598,Seattle,SOCW,Social Work,598,Integrative Seminar,"1-5, max. 12",NaN,NaN,NaN,NaN,NaN,"Integrative, skill-based seminar that provides...",SOCW
15679,18844,SOCW599,Seattle,SOCW,Social Work,599,Independent Study in Social Work,"1-10, max. 10",NaN,NaN,NaN,NaN,NaN,Prerequisite: approval of a well-specified pla...,SOCW
15680,18845,SOCW600,Seattle,SOCW,Social Work,600,Independent Study or Research,"1-10, max. 10",NaN,NaN,NaN,SOCW600,NaN,An independent research project. Prerequisite:...,SOCW


In [4]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# URL of UW Spring 2025 Time Schedule
url = "https://www.washington.edu/students/timeschd/SPR2025/"

# Fetch page content
response = requests.get(url)
response.raise_for_status()  # Ensure successful request

# Parse HTML
soup = BeautifulSoup(response.text, "html.parser")

# Extract text content
text = soup.get_text()

# Regex to capture text and abbreviations in parentheses
matches = re.findall(r'([A-Za-z &,\-]+?)\s*\(([^)]+)\)', text)

# Convert to DataFrame
abbreviation_df = pd.DataFrame(matches, columns=["Department Name", "Abbreviation"])
abbreviation_df['Short Abbreviation'] = abbreviation_df['Abbreviation'].replace(" ", "")

# Display results


In [5]:
abbreviation_df

,Department Name,Abbreviation,Short Abbreviation
0,,"Last updated February 17, 2025.","Last updated February 17, 2025."
1,The University reserves the right to,1,1
2,,2,2
3,,3,3
4,,4,4
...,...,...,...
376,Naval Science,N SCI,N SCI
377,Social Welfare BASW,SOC WF,SOC WF
378,Social Welfare,SOC WL,SOC WL
379,Social Work,MSW,MSW


In [6]:
merged_classes = pd.merge(classes, abbreviation_df, left_on= 'Prefix', right_on='Short Abbreviation')


In [7]:
merged_classes

,Unnamed: 0,Course ID,Campus,Department Name_x,College,Course Number,Course Name,Credits,Areas of Knowledge,Quarters Offered,Offered with,Prerequisites,Co-Requisites,Description,Prefix,Department Name_y,Abbreviation,Short Abbreviation
0,3165,SWA101,Seattle,SWA,College of Arts and Sciences,101,Basic Swahili,5,NaN,NaN,NaN,NaN,NaN,Introduces the Swahili language and the divers...,SWA,Swahili,SWA,SWA
1,3166,SWA102,Seattle,SWA,College of Arts and Sciences,102,Basic Swahili,5,NaN,NaN,NaN,SWA101,NaN,Introduces the Swahili language and the divers...,SWA,Swahili,SWA,SWA
2,3167,SWA103,Seattle,SWA,College of Arts and Sciences,103,Basic Swahili,5,NaN,NaN,NaN,SWA102,NaN,Introduces the Swahili language and the divers...,SWA,Swahili,SWA,SWA
3,3168,SWA134,Seattle,SWA,College of Arts and Sciences,134,Intensive Basic Swahili,15,NaN,S,NaN,NaN,NaN,Introduces the Swahili language and the divers...,SWA,Swahili,SWA,SWA
4,3169,SWA201,Seattle,SWA,College of Arts and Sciences,201,Intermediate Swahili,5,NaN,NaN,NaN,"SWA103,SWA134",NaN,Builds proficiency in the language by speaking...,SWA,Swahili,SWA,SWA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13038,18749,HSERV598,Seattle,HSERV,School of Public Health,598,Public Health Integrative Learning Experience,4,NaN,NaN,NaN,NaN,NaN,Students create a culminating project for the ...,HSERV,Health Services,HSERV,HSERV
13039,18750,HSERV599,Seattle,HSERV,School of Public Health,599,Capstone Project,*-,NaN,NaN,NaN,NaN,NaN,Applies and extends the public health skills l...,HSERV,Health Services,HSERV,HSERV
13040,18751,HSERV600,Seattle,HSERV,School of Public Health,600,Independent Study or Research,*-,NaN,NaN,NaN,POI,NaN,Prerequisite: permission of instructor.,HSERV,Health Services,HSERV,HSERV
13041,18752,HSERV700,Seattle,HSERV,School of Public Health,700,Master's Thesis,*-,NaN,NaN,NaN,POI,NaN,Prerequisite: permission of instructor.,HSERV,Health Services,HSERV,HSERV


In [8]:
import requests

# Cookies for authentication (ensure these are valid)
cookies = {
    "_ga": "GA1.1.1169709652.1732947157",
    "_ga_0V5LFWD2KQ": "GS1.1.1732947156.1.1.1732947225.57.0.0",
    "_hp2_id.3001039959": "%7B%22userId%22%3A%223766466782283369%22%2C%22pageviewId%22%3A%22836061124374915%22%2C%22sessionId%22%3A%224498099947331590%22%2C%22identity%22%3A%22uu-2-6523be9528e79cfc59958d23c6f56a841a282a84817f28115c28924b5dc768ad-hMdW75SAnZRIzmj4yn9Zl1cW9eAMfEUMSztnrw0F%22%2C%22trackerVersion%22%3A%224.0%22%2C%22identityField%22%3Anull%2C%22isIdentified%22%3A1%7D",
    "_hp2_props.3001039959": "%7B%22Base.appName%22%3A%22Canvas%22%7D",
    "csrftoken": "q26ZTcxt7iiGfFpcv5D0BrJGJocKLhbb",
    "sessionid": "tiu0wipzn0zfodr1t5shg4kbto05txed"
}

# Data storage
catalog = {}
missed_classes = []  # Track missed class fetches
processed_classes = set()  # Track processed class numbers

def fetch_class_details(row):
    """Fetch class details from the API using the Abbreviation column."""
    class_num = row['Course Number']
    abbreviation = row['Abbreviation']

    class_key = f"{abbreviation}{class_num}"

    # Replace spaces in abbreviation with %20 for URL formatting
    formatted_abbreviation = "%20".join(abbreviation.split())

    # Construct the URL
    url = f"https://dawgpath.uw.edu/api/v1/courses/details/{formatted_abbreviation}%20{class_num}"

    # Log request attempt
    print(f"Attempting to fetch: {formatted_abbreviation} {class_num}")

    try:
        response = requests.get(url, cookies=cookies)
        processed_classes.add(class_num)  # Mark as processed

        if response.status_code == 200:
            try:
                data = response.json()
            except ValueError:
                data = response.text  # If response isn't JSON, return text
                print(f"Warning: Non-JSON response for {class_num}")

            return class_key, data

        else:
            print(f"Error {response.status_code} for {class_num} - {url}")
            missed_classes.append({"class": class_num, "error": f"HTTP {response.status_code}", "abbreviation": abbreviation})
            return class_num, None

    except Exception as e:
        print(f"Request failed for {class_num}: {e}")
        missed_classes.append({"class": class_num, "error": str(e), "abbreviation": abbreviation})
        return class_key, None

# Sequentially process all rows (no threading)
for _, row in merged_classes.iterrows():
    class_key, data = fetch_class_details(row)
    if data is not None:
        catalog[class_key] = data  # Store results in dictionary
    else:
        missed_classes.append({"class": class_key, "error": "No data", "abbreviation": row["Abbreviation"]})

# Find missing entries
all_classes = set(merged_classes["Course Number"].unique())
unprocessed_classes = all_classes - processed_classes

# Print Summary
print("\n=== Summary ===")
print(f"✅ Successfully fetched {len(catalog)} classes.")
print(f"❌ Failed to fetch {len(missed_classes)} classes.")

if unprocessed_classes:
    print(f"⚠️ Classes not processed at all: {unprocessed_classes}")


Attempting to fetch: SWA 101
Attempting to fetch: SWA 102
Attempting to fetch: SWA 103
Attempting to fetch: SWA 134
Attempting to fetch: SWA 201
Attempting to fetch: SWA 202
Attempting to fetch: SWA 203
Attempting to fetch: ARCTIC 101
Attempting to fetch: ARCTIC 102
Attempting to fetch: ARCTIC 103
Attempting to fetch: ARCTIC 200
Attempting to fetch: ARCTIC 201
Attempting to fetch: ARCTIC 202
Attempting to fetch: ARCTIC 203
Attempting to fetch: ARCTIC 210
Error 404 for 210 - https://dawgpath.uw.edu/api/v1/courses/details/ARCTIC%20210
Attempting to fetch: ARCTIC 301
Attempting to fetch: ARCTIC 302
Attempting to fetch: ARCTIC 303
Attempting to fetch: ARCTIC 308
Attempting to fetch: ARCTIC 321
Attempting to fetch: ARCTIC 387
Attempting to fetch: ARCTIC 391
Attempting to fetch: ARCTIC 400
Attempting to fetch: ARCTIC 401
Attempting to fetch: ARCTIC 410
Error 404 for 410 - https://dawgpath.uw.edu/api/v1/courses/details/ARCTIC%20410
Attempting to fetch: ARCTIC 498
Attempting to fetch: TAGLG 10

In [9]:
missed_classes

[{'class': '210', 'error': 'HTTP 404', 'abbreviation': 'ARCTIC'},
 {'class': '210', 'error': 'No data', 'abbreviation': 'ARCTIC'},
 {'class': '410', 'error': 'HTTP 404', 'abbreviation': 'ARCTIC'},
 {'class': '410', 'error': 'No data', 'abbreviation': 'ARCTIC'},
 {'class': '500', 'error': 'HTTP 404', 'abbreviation': 'CFRM'},
 {'class': '500', 'error': 'No data', 'abbreviation': 'CFRM'},
 {'class': '501', 'error': 'HTTP 404', 'abbreviation': 'CFRM'},
 {'class': '501', 'error': 'No data', 'abbreviation': 'CFRM'},
 {'class': '502', 'error': 'HTTP 404', 'abbreviation': 'CFRM'},
 {'class': '502', 'error': 'No data', 'abbreviation': 'CFRM'},
 {'class': '503', 'error': 'HTTP 404', 'abbreviation': 'CFRM'},
 {'class': '503', 'error': 'No data', 'abbreviation': 'CFRM'},
 {'class': '504', 'error': 'HTTP 404', 'abbreviation': 'CFRM'},
 {'class': '504', 'error': 'No data', 'abbreviation': 'CFRM'},
 {'class': '505', 'error': 'HTTP 404', 'abbreviation': 'CFRM'},
 {'class': '505', 'error': 'No data', '

5777

In [11]:
with open("catalog.json", "w") as json_file:
    json.dump(catalog, json_file, indent=4)  # `indent=4` makes it more readable